In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [16]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [17]:
df_train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,29.4846,5.8683,3.8208,15.8348,-5.0121,15.1345,3.2003,9.3192,3.8821,5.7999,5.5378,5.0988,22.0330,5.5134,30.2645,10.4968,-7.2352,16.5721,-7.3477,11.0752,-5.5937,9.4878,-14.9100,9.4245,22.5441,-4.8622,7.6543,-15.9319,13.3175,-0.3566,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,13.2070,5.8442,4.7086,5.7141,-1.0410,20.5092,3.2790,-5.5952,7.3176,5.7690,-7.0927,-3.9116,7.2569,-5.8234,25.6820,10.9202,-0.3104,8.8438,-9.7009,2.4013,-4.2935,9.3908,-13.2648,3.1545,23.0866,-5.3000,5.3745,-6.2660,10.1934,-0.8417,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,31.8833,5.9684,7.2084,3.8899,-11.0882,17.2502,2.5881,-2.7018,0.5641,5.3430,-7.1541,-6.1920,18.2366,11.7134,14.7483,8.1013,11.8771,13.9552,-10.4701,5.6961,-3.7546,8.4117,1.8986,7.2601,-0.4639,-0.0498,7.9336,-12.8279,12.4124,1.8489,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,33.5107,5.6953,5.4663,18.2201,6.5769,21.2607,3.2304,-1.7759,3.1283,5.5518,1.4493,-2.6627,19.8056,2.3705,18.4685,16.3309,-3.3456,13.5261,1.7189,5.1743,-7.6938,9.7685,4.8910,12.2198,11.8503,-7.8931,6.4209,5.9270,16.0201,-0.2829,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [18]:
train_df = df_train.copy()
test_df = df_test.copy()
train_df.drop(columns=["ID_code", "target"], inplace=True)
test_df.drop(columns=["ID_code"], inplace=True)
target = df_train.target

In [19]:
def augment_train(df_train, y_train):   
    t0 = df_train[y_train == 0].copy()
    t1 = df_train[y_train == 1].copy()
    i = 0
    N = 3
    for I in range(2):  # augment data into 2x
        for col in df_train.columns:
            i = i + 1000
            np.random.seed(i)
            np.random.shuffle(t0[col].values)
            np.random.shuffle(t1[col].values)
        df_train = pd.concat([df_train, t0.copy()])
        df_train = pd.concat([df_train, t1.copy()])
        y_train = pd.concat([y_train, pd.Series([0] * t0.shape[0]), pd.Series([1] * t1.shape[0])])
    return df_train, y_train

In [21]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
from catboost import Pool, CatBoostClassifier

model = CatBoostClassifier(subsample=0.36, #rawdata 0.5  ×2 0.45 ×3 0.36
                            custom_loss='Logloss',
                           random_strength = 0,
                           max_depth=3,
                           eval_metric="AUC",
                           learning_rate=0.01,
                           iterations=60000,
                           #class_weights=[1,2],
                           bootstrap_type='Bernoulli',
                           #rsm=0.045,
                           l2_leaf_reg=0.3,
                           task_type="GPU",
                           random_seed=432013,
                           od_type="Iter",
                           border_count=128
                           #has_time= True 
                          )

In [22]:
def run_cat(model,  trt, tst, tar,n_splits=11, plot=False):   
    kf = KFold(n_splits=n_splits, random_state=432013, shuffle=True)
    oof = np.zeros(len(trt))
    feature_importance_df = pd.DataFrame()
    y_valid_pred = 0 * tar
    y_test_pred = 0
    for n_fold, (train_index, valid_index) in enumerate(kf.split(trt, tar)):
        y_train, y_valid = tar.iloc[train_index], tar.iloc[valid_index]
        X_train, X_valid = trt.iloc[train_index,:], trt.iloc[valid_index,:]
        X_train, y_train = augment_train(X_train, y_train)
        _train = Pool(X_train, label=y_train)
        _valid = Pool(X_valid, label=y_valid)
        print( "Fold ", n_fold)
        fit_model = model.fit(_train,
                              verbose_eval=1000, 
                              early_stopping_rounds=1000,
                              eval_set=[_valid],
                              use_best_model=True,
                              plot=False,
                                            
                             )
        pred = fit_model.predict_proba(X_valid)[:,1]
        oof[valid_index] = pred
        print( "auc = ", roc_auc_score(y_valid, pred) )
        y_valid_pred.iloc[valid_index] = pred
        y_test_pred += fit_model.predict_proba(tst)[:,1]
    y_test_pred /= n_splits
    print("average auc:", roc_auc_score(tar, oof))
    return y_test_pred, oof

In [23]:
y_test_pred, oof = run_cat(model,train_df, test_df, target)

Fold  0
0:	learn: 0.6092861	test: 0.6016245	best: 0.6016245 (0)	total: 12.4ms	remaining: 12m 25s
1000:	learn: 0.8305681	test: 0.8252141	best: 0.8252141 (1000)	total: 10.6s	remaining: 10m 24s
2000:	learn: 0.8611386	test: 0.8548256	best: 0.8548256 (2000)	total: 21.2s	remaining: 10m 13s
3000:	learn: 0.8762776	test: 0.8692761	best: 0.8692812 (2999)	total: 31.6s	remaining: 10m 1s
4000:	learn: 0.8852909	test: 0.8779186	best: 0.8779186 (4000)	total: 42.3s	remaining: 9m 52s
5000:	learn: 0.8908671	test: 0.8831933	best: 0.8831933 (5000)	total: 52.8s	remaining: 9m 40s
6000:	learn: 0.8950517	test: 0.8872102	best: 0.8872102 (6000)	total: 1m 3s	remaining: 9m 31s
7000:	learn: 0.8981479	test: 0.8900816	best: 0.8900827 (6999)	total: 1m 14s	remaining: 9m 21s
8000:	learn: 0.9005518	test: 0.8921365	best: 0.8921386 (7998)	total: 1m 25s	remaining: 9m 13s
9000:	learn: 0.9024647	test: 0.8939074	best: 0.8939077 (8998)	total: 1m 35s	remaining: 9m 2s
10000:	learn: 0.9039852	test: 0.8951954	best: 0.8951967 (9998)

15000:	learn: 0.9088084	test: 0.8972289	best: 0.8972323 (14997)	total: 2m 41s	remaining: 8m 4s
16000:	learn: 0.9094059	test: 0.8975312	best: 0.8975386 (15942)	total: 2m 52s	remaining: 7m 54s
17000:	learn: 0.9099342	test: 0.8978860	best: 0.8978863 (16998)	total: 3m 3s	remaining: 7m 43s
18000:	learn: 0.9104297	test: 0.8981163	best: 0.8981203 (17984)	total: 3m 13s	remaining: 7m 32s
19000:	learn: 0.9108972	test: 0.8983970	best: 0.8983970 (19000)	total: 3m 24s	remaining: 7m 21s
20000:	learn: 0.9113306	test: 0.8985548	best: 0.8985548 (20000)	total: 3m 36s	remaining: 7m 12s
21000:	learn: 0.9117354	test: 0.8987434	best: 0.8987434 (21000)	total: 3m 47s	remaining: 7m 1s
22000:	learn: 0.9121308	test: 0.8989124	best: 0.8989149 (21954)	total: 3m 58s	remaining: 6m 51s
23000:	learn: 0.9125080	test: 0.8990362	best: 0.8990379 (22998)	total: 4m 9s	remaining: 6m 41s
24000:	learn: 0.9128715	test: 0.8991011	best: 0.8991023 (23998)	total: 4m 20s	remaining: 6m 30s
25000:	learn: 0.9132200	test: 0.8992184	best

11000:	learn: 0.9049353	test: 0.8974627	best: 0.8974661 (10990)	total: 1m 58s	remaining: 8m 47s
12000:	learn: 0.9059389	test: 0.8982750	best: 0.8982750 (12000)	total: 2m 9s	remaining: 8m 36s
13000:	learn: 0.9067874	test: 0.8989104	best: 0.8989104 (12999)	total: 2m 19s	remaining: 8m 25s
14000:	learn: 0.9075261	test: 0.8994691	best: 0.8994702 (13999)	total: 2m 30s	remaining: 8m 15s
15000:	learn: 0.9081806	test: 0.8998742	best: 0.8998751 (14999)	total: 2m 41s	remaining: 8m 5s
16000:	learn: 0.9087543	test: 0.9002520	best: 0.9002535 (15999)	total: 2m 52s	remaining: 7m 55s
17000:	learn: 0.9092835	test: 0.9005651	best: 0.9005657 (16999)	total: 3m 3s	remaining: 7m 44s
18000:	learn: 0.9097611	test: 0.9008006	best: 0.9008030 (17996)	total: 3m 14s	remaining: 7m 34s
19000:	learn: 0.9102092	test: 0.9010166	best: 0.9010214 (18990)	total: 3m 25s	remaining: 7m 23s
20000:	learn: 0.9106301	test: 0.9012119	best: 0.9012123 (19998)	total: 3m 36s	remaining: 7m 12s
21000:	learn: 0.9110239	test: 0.9013901	bes

12000:	learn: 0.9061971	test: 0.8971023	best: 0.8971023 (12000)	total: 2m 10s	remaining: 8m 41s
13000:	learn: 0.9070626	test: 0.8976328	best: 0.8976328 (13000)	total: 2m 21s	remaining: 8m 30s
14000:	learn: 0.9078161	test: 0.8981134	best: 0.8981134 (14000)	total: 2m 31s	remaining: 8m 19s
15000:	learn: 0.9084959	test: 0.8985208	best: 0.8985219 (14997)	total: 2m 42s	remaining: 8m 8s
16000:	learn: 0.9090717	test: 0.8988305	best: 0.8988346 (15997)	total: 2m 54s	remaining: 7m 58s
17000:	learn: 0.9096054	test: 0.8990747	best: 0.8990756 (16998)	total: 3m 5s	remaining: 7m 49s
18000:	learn: 0.9100974	test: 0.8992260	best: 0.8992307 (17950)	total: 3m 16s	remaining: 7m 39s
19000:	learn: 0.9105673	test: 0.8994066	best: 0.8994095 (18996)	total: 3m 27s	remaining: 7m 28s
20000:	learn: 0.9110146	test: 0.8995136	best: 0.8995142 (19997)	total: 3m 38s	remaining: 7m 16s
21000:	learn: 0.9114276	test: 0.8996180	best: 0.8996180 (21000)	total: 3m 49s	remaining: 7m 5s
22000:	learn: 0.9118152	test: 0.8997113	bes

30000:	learn: 0.9136804	test: 0.9041468	best: 0.9041620 (29819)	total: 5m 26s	remaining: 5m 26s
31000:	learn: 0.9139847	test: 0.9041880	best: 0.9041999 (30887)	total: 5m 37s	remaining: 5m 16s
32000:	learn: 0.9142915	test: 0.9042333	best: 0.9042398 (31730)	total: 5m 49s	remaining: 5m 5s
bestTest = 0.9042398334
bestIteration = 31730
Shrink model to first 31731 iterations.
auc =  0.9042399619867842
Fold  10
0:	learn: 0.6099007	test: 0.6086443	best: 0.6086443 (0)	total: 12.5ms	remaining: 12m 32s
1000:	learn: 0.8313645	test: 0.8231233	best: 0.8231233 (1000)	total: 10.9s	remaining: 10m 42s
2000:	learn: 0.8615699	test: 0.8527045	best: 0.8527045 (2000)	total: 21.7s	remaining: 10m 29s
3000:	learn: 0.8767934	test: 0.8682073	best: 0.8682073 (3000)	total: 32.6s	remaining: 10m 20s
4000:	learn: 0.8854805	test: 0.8774511	best: 0.8774511 (4000)	total: 44s	remaining: 10m 16s
5000:	learn: 0.8910411	test: 0.8832854	best: 0.8832854 (5000)	total: 54.6s	remaining: 10m
6000:	learn: 0.8950406	test: 0.8875722	

In [24]:
submission = pd.read_csv("../input/sample_submission.csv")
submission['target'] = y_test_pred
pd.Series(oof).to_csv("Cat_oof.csv", index = False)
submission.to_csv('submission.csv', index=False)